In [ ]:
docker run -i -t -v /data/data1/:/data -v /home:/hm hub.ai.xm.gov.cn/comp_120/geopandas /bin/bash 
clear
python


In [ ]:
DATA_PATH = '/data/'
DATA_TAXI_ORDER = DATA_PATH+'taxiOrder/'
DATA_FILE_NAME = 'taxiOrder201951-54.csv'
# DATA_FILE_NAME = 'taxiOrder_2018429-501.csv'
DATA_FILE_OUTPUT_NAME = '/xm/gdf2018429-501.csv'
# DATA_FILE_OUTPUT_NAME = '/xm/gdf201951-54.csv'

In [ ]:
import pandas as pd
import datetime



df = pd.read_csv(DATA_TAXI_ORDER+DATA_FILE_NAME)
total_num = len(df)

df.dropna(subset=['GETON_LONGITUDE', 
                  'GETON_LATITUDE', 
                  'GETOFF_LONGITUDE', 
                  'GETOFF_LATITUDE',
                  'PASS_MILE',
                  'CAR_NO'], inplace=True)
na_num = len(df)

df['GETON_DATE'] = pd.to_datetime(df['GETON_DATE'])
df['GETOFF_DATE'] = pd.to_datetime(df['GETOFF_DATE'])

df['trip_duration'] = (df['GETOFF_DATE'] - df['GETON_DATE']).dt.total_seconds()/60
df = df[df['trip_duration'] > 1]
more_then_1_num = len(df)

# filter data 
lat_range = [24.2106, 25.0881]
long_range = [117.3944, 118.8913]
df_filtered = df[
    (df['GETON_LONGITUDE'] > long_range[0])
    & (df['GETON_LONGITUDE'] < long_range[1])
    & (df['GETON_LATITUDE'] > lat_range[0])
    & (df['GETON_LATITUDE'] < lat_range[1])
    & (df['GETOFF_LONGITUDE'] > long_range[0])
    & (df['GETOFF_LONGITUDE'] < long_range[1])
    & (df['GETOFF_LATITUDE'] > lat_range[0])
    & (df['GETOFF_LATITUDE'] < lat_range[1])
    & (df['trip_duration'] > 0.5)
    & (df['trip_duration'] < 37) # 34
]
in_range_num = len(df)

df_filtered['day'] = df_filtered['GETON_DATE'].apply(
    lambda dt: datetime.datetime(dt.year, dt.month, dt.day)
)

df_filtered['hour'] = df_filtered['GETON_DATE'].apply(
    lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour)
)
# df_filtered['off_hour'] = df_filtered['GETOFF_DATE'].dt.hour



import geopandas as gpd

gdf = gpd.GeoDataFrame(
    df_filtered, geometry=gpd.points_from_xy(df_filtered['GETON_LATITUDE'],df_filtered['GETON_LONGITUDE'])
)

import geopandas as gpd
from shapely.geometry import Point

df_geometry = [Point(xy) for xy in zip(df_filtered.GETON_LONGITUDE, df_filtered.GETON_LATITUDE)]
gdf = gpd.GeoDataFrame(df_filtered, geometry=df_geometry)
gdf.crs={'init':'epsg:4326'}


import pandas as pd
import datetime
import numpy as np
import pytz   
import math
import geopandas as gpd
from shapely.geometry import Point,Polygon
from fiona.crs import from_epsg


lat_range = [24.2106, 25.0881]
long_range = [117.3944, 118.8913]

y = np.linspace(lat_range[0],lat_range[1],18)
x = np.linspace(long_range[0],long_range[1],18)

grid = []
for m,i in enumerate(x[:17]):
    for k,j in enumerate(y[:17]):
        cord = np.array([i,j,x[m+1],j,x[m+1],y[k+1],i,y[k+1]]).reshape(4,2)
        grid.append(Polygon(cord))


area = gpd.GeoDataFrame(grid)
area = area.reset_index()
area.columns=['index','geometry']
area.crs={'init':'epsg:4326'}



t_gdf = gpd.sjoin(gdf, area, op='within', how='left')
t_gdf.to_csv(DATA_FILE_OUTPUT_NAME, index=False)






In [ ]:
import pandas as pd
import datetime
import numpy as np
import pytz   
import math
import geopandas as gpd
from shapely.geometry import Point,Polygon
from fiona.crs import from_epsg

t_gdf = pd.read_csv(DATA_FILE_OUTPUT_NAME)
t_gdf['hour'] = pd.to_datetime(t_gdf['hour'])


lat_range = [24.2106, 25.0881]
long_range = [117.3944, 118.8913]

y = np.linspace(lat_range[0],lat_range[1],18)
x = np.linspace(long_range[0],long_range[1],18)

grid = []
for m,i in enumerate(x[:17]):
    for k,j in enumerate(y[:17]):
        cord = np.array([i,j,x[m+1],j,x[m+1],y[k+1],i,y[k+1]]).reshape(4,2)
        grid.append(Polygon(cord))


area = gpd.GeoDataFrame(grid)
area = area.reset_index()
area.columns=['index','geometry']
area.crs={'init':'epsg:4326'}

t_stat = area.loc[:,['index']]

l_l_c = []
for key, item in t_gdf.groupby('index'):
    t_dict = {}
    for k, it in item.groupby('hour'):
        t_dict[int(k.strftime('%s'))] = len(it)
    l_l_c.append([int(key)]+[t_dict[i] for i in sorted(t_dict, key=t_dict.get)])


t_df = pd.DataFrame(l_l_c, columns=['index']+[x for x in range(0,24*3)])


In [ ]:
l_l_c = []
for key, item in t_gdf.groupby('index'):
    t_dict = {}
    for k, it in item.groupby('hour'):
        t_dict[k.dt.total_seconds()] = len(it)
    l_l_c.append([t_dict[i] for i in sorted(t_dict, key=t_dict.get)])

In [ ]:
# t_stat = pd.merge(t_stat,pd.DataFrame(l_l_c))

t_stat = pd.merge(t_stat, t_gdf.groupby('index')['hour'].count().to_frame().reset_index(),
               on='index', how='outer')
t_stat.columns = ['index']+[i for i in range(0,24)]
t_stat.fillna(0,inplace=True) 

In [ ]:
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

series = gpd.read_file(DATA_FILE_OUTPUT_NAME)

In [ ]:
X = series.values
size = int(len(X)-24)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
for t in range(len(test)):
    model = ARIMA(history, order=(5,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

In [ ]:
import ast
from operator import add

In [ ]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
## split dataset into training and test 
train = all_valuse[:len(df_train), :]
test = all_valuse[len(df_train):, :]

In [ ]:
df_train.loc[df_train['Day']=="2018-04-29 00:00:00", 'Day'] = 1
df_train.loc[df_train['Day']=="2018-04-30 00:00:00", 'Day'] = 2
df_train.insert(0, 'count', df_train.pop('count'))

df_test.loc[df_test['Day']=="2018-05-01 00:00:00", 'Day'] = 3
df_test.insert(0, 'count', df_test.pop('count'))

In [ ]:
# split into input and outputs for model
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

# reshape input to samples, time and features
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='rmse', optimizer='adam', metrics=['accuracy'])
# fit network
history = model.fit(train_X, train_y, epochs=10, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

In [ ]:
import numpy as np 
import pandas as pd
import datetime as dt
from sklearn.model_selection import train_test_split
import xgboost as xgb
import os
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#Cross-validation
params = {
    # 参数设计.
    'max_depth': 15
    'eta':.03, 
    'subsample': 1, 
    'colsample_bytree': 0.8,
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'silent': 1
}

CV=False
if CV:
    dtrain = xgb.DMatrix(train,label=y)
    gridsearch_params = [
        (eta)
        for eta in np.arange(.04, 0.12, .02)
    ]

    # Define initial best params and RMSE
    min_rmse = float("Inf")
    best_params = None
    for (eta) in gridsearch_params:
        print("CV with eta={} ".format(
                                 eta))

        # Update our parameters
        params['eta'] = eta

        # Run CV
        cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=1000,
            nfold=3,
            metrics={'rmse'},
            early_stopping_rounds=10
        )

        # Update best RMSE
        mean_rmse = cv_results['test-rmse-mean'].min()
        boost_rounds = cv_results['test-rmse-mean'].argmin()
        print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
        if mean_rmse < min_rmse:
            min_rmse = mean_rmse
            best_params = (eta)

    print("Best params: {}, RMSE: {}".format(best_params, min_rmse))
else:
    #Print final params to use for the model
    params['silent'] = 0 #Turn on output
    print(params)

In [ ]:
## XGBoost模型
def XGBmodel(x_train,x_test,y_train,y_test,params):
    matrix_train = xgb.DMatrix(x_train,label=y_train)
    matrix_test = xgb.DMatrix(x_test,label=y_test)
    model=xgb.train(params=params,
                    dtrain=matrix_train,num_boost_round=2000, 
                    early_stopping_rounds=100,evals=[(matrix_test,'test')])
    return model

model = XGBmodel(x_train,x_test,y_train,y_test,params)